In [3]:
import json

class Args:
    min_user_freq = 10
    min_book_freq = 10
    max_user_freq = 200

args = Args()


In [2]:
def get_valid_user_and_book_set():
    user_freq, book_freq = {}, {}
    with open('data/ratings_Books.csv', 'r') as f:
        for line in f.readlines():
            # user, item, rating, timestamp
            splitted = line.split(",")
            try:
                user_freq[splitted[0]] += 1
            except KeyError:
                user_freq[splitted[0]] = 1
            try:
                book_freq[splitted[1]] += 1
            except KeyError:
                book_freq[splitted[1]] = 1

    valid_users = set([user_id for user_id, cnt in user_freq.items() if cnt >= args.min_user_freq and cnt <= args.max_user_freq])
    valid_books = set([book_id for book_id, cnt in book_freq.items() if cnt >= args.min_book_freq])

    return valid_users, valid_books


def parse_user_data(valid_users, valid_books):
    user_ratings = {}
    with open('data/ratings_Books.csv', 'r') as f:
        for line in f.readlines():
            line = line[:-1]
            splitted = line.split(",")
            user_id, book_id = splitted[0], splitted[1]
            if user_id in valid_users and book_id in valid_books:
                try:
                    # not using rating here
                    user_ratings[user_id].append((book_id, int(splitted[-1])))
                except KeyError:
                    user_ratings[user_id] = [(book_id, int(splitted[-1]))]

    return user_ratings


valid_users, valid_books = get_valid_user_and_book_set()
user_ratings = parse_user_data(valid_users, valid_books)


In [8]:
def read_book_meta(valid_books):
    book_cates = {}
    with open("data/meta_Books.json", "r") as f:
        for line in f.readlines():
            meta = json.loads(line[:-1])
            if meta['asin'] in valid_books:
                book_cates[meta['asin']] = meta['category']

    return book_cates

book_cates = read_book_meta(valid_books)



In [ ]:
def encode_user

1. Each rating action reflect user's interest upon that book, so each rating action is one postive sample.
2. Split all the rating actions to 19:1 as described in the paper.
3. From each rating action, use that user's rating actions before the current rating time as history behaviors.
4. For negative samples, just randomly draw from the whole book pool.

['0000230022',
 '000100039X',
 '0001473727',
 '0001472933',
 '0001473123',
 '000171287X',
 '0001711296',
 '0001714384',
 '0001954563',
 '0002005395']

In [16]:
book_cates['000171287X']

['Books', "Children's Books", 'Literature & Fiction']